# Working bits from bpstats09pg


## moving to sqlite after trouble with queries and sequences in postgresql

https://docs.sqlalchemy.org/en/14/dialects/sqlite.html#module-sqlalchemy.dialects.sqlite.pysqlite

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

use %%black to format cell

running from fedora and making change to see if push works with flatboy
can't seem to get gitlab to show bubba as an author


In [ ]:
import dateutil
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import select
from sqlalchemy.dialects.sqlite import DATETIME as DT
from sqlalchemy.orm import sessionmaker
import matplotlib.pyplot as plt
# import seaborn as sb
from matplotlib import cbook, dates
from icecream import ic
#from matplotlib.ticker import Formatter
import mplcursors
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate, Table
from reportlab.platypus import TableStyle
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
# %matplotlib widget
# this gives seperate window qt widget for figures
%matplotlib qt5
# %matplotlib inline
# %load_ext blackcellmagic
eng = create_engine("sqlite:////data/sqlite/vitals.db")
myconn = eng.connect()

## Blood sugar line plot

In [ ]:
myconn

In [ ]:
c = canvas.Canvas("reportlab_pdf.pdf")
c.drawString(100,100,"Hello World")
c.showPage()
c.save()

In [ ]:
#  this one working and showing the variable mystats n
# never got cursor to work so added mplcursors lib
# get data
sugeightdays = "SELECT bsdate,bsugar FROM qtsugar WHERE bsdate > (SELECT date('now','-8 day'))"
sugar8days = pd.read_sql_query(sugeightdays, myconn, parse_dates = "bsdate")
myconn
# start setting up figure
mylegend = "7 days stats "
mystats = sugar8days.describe(include='int')
sugar8days=sugar8days.sort_values("bsdate")
fig3, ax3 = plt.subplots()
plt.ylim(60,150)
ax3.set_xlabel('Date')
plt.title('blood sugar last 8 days')
ax3.annotate([mystats], xy=(200, 380), xycoords='figure points')
plt.setp(ax3.get_xticklabels(), rotation = 90, fontsize=6)
#ax3.set_xticklabels(sugar8days.bsdate, rotation=90, fontsize=6)
#plt.grid(b=True, which='both', axis='both', )
plt.grid
fig3.set_figwidth(10)
fig3.set_figheight(10)
lines = ax3.plot(sugar8days.bsdate , sugar8days.bsugar, marker='o', linestyle='dashed' )
mplcursors.cursor(lines) # or just mplcursors.cursor()



In [ ]:
sugar8days.describe()

In [ ]:
sugar8days.pdf

## Blood sugar line plot and table

In [ ]:
# this one now working and showing 2 subplots Saturday, June 5, 2021 10:57:39 AM EDT
# never got cursor to work so added mplcursors lib
# get data
sugeightdays = (
    "SELECT bsdate,bsugar FROM qtsugar WHERE bsdate > (SELECT date('now','-8 day'))"  # pgsql
)
sugar8days = pd.read_sql_query(sugeightdays, myconn, parse_dates="bsdate")
fig3, ax3, = plt.subplots(1, 2)
plt.subplot(1, 2, 1)
# start setting up figure
mylegend = "7 days stats "
mystats = sugar8days.describe()
plt.ylim(60, 150)
ax3[0].set_xlabel("Date")
plt.title("blood sugar last 8 days")
ax3[0].annotate([mystats], xy=(200, 380), xycoords="figure points")
plt.setp(ax3[0].get_xticklabels(), rotation=90, fontsize=6)
# ax3.set_xticklabels(sugar8days.bsdate, rotation=90, fontsize=6)
plt.grid(
    b=True, which="both", axis="both",
)
fig3.set_figwidth(15)
fig3.set_figheight(10)
# thetable = pd.plotting.table(fig3, sugar8days, colLabels= sugar8days.columns )
lines = ax3[0].plot(
    sugar8days.bsdate, sugar8days.bsugar, marker="o", linestyle="dashed"
)
plt.subplot(1, 2, 2)

plt.table(
    cellText=sugar8days.values,
    colWidths=[0.25] * len(sugar8days.columns),
    rowLabels=None,
    colLabels=sugar8days.columns,
    cellLoc="center",
    rowLoc="center",
    loc="center",
)
mplcursors.cursor(lines)  # or just mplcursors.cursor()
plt.show()

## Blood pressure

In [ ]:
bpsevendays = "SELECT bpdate ,bpsys AS systolic, bpdia AS diastolic, bphr AS pulse from  vsigns_bp where bpdate > (SELECT date('now','-8 day'))"
bp7days = pd.read_sql_query(bpsevendays, myconn, parse_dates = "bpdate")

x = np.arange(len(bp7days))  # the label locations
width = 0.35  # the width of the bars

# color picker gpick
fig , ax = plt.subplots()
rects1 = ax.bar(x - width/2, bp7days.systolic, width, label='systolic', facecolor='#00388F')
rects2 = ax.bar(x + width/2, bp7days.diastolic, width, label='diastolic', facecolor='#8F5600')
rects3 = ax.bar(x + width, bp7days.pulse, width, label='pulse', facecolor='#638F00')
ax.axhline(y=120, color='#00388F')
ax.axhline(y=80, color='#8F5600')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('120/80 = perfect')
ax.set_title('Blood press 7 days')
ax.set_xticks(x)
ax.set_xticklabels(bp7days.bpdate, rotation=90, fontsize=6)
ax.legend()

ax.bar_label(rects1, label_type='center', color='#EEEED0')
ax.bar_label(rects2, label_type='center')
ax.bar_label(rects3, label_type='center')
plt.rc('xtick', labelsize=6)
fig.set_figheight(10)
mplcursors.cursor(rects3)
mplcursors.cursor(rects2)
plt.show()

In [ ]:
bp7days.describe()

## Weight line plot and table

In [ ]:
# editied in kate for new query for weight (fatty table)
# never got cursor to work so added mplcursors lib
# get data
# wtdta = "select ftime, weight from qfatty where ftime > (SELECT date('now','-30 day'))" # original 14 days
wtdta = "select ftime, weight from (select ftime, weight from fatty order by ftime desc limit 30) order by ftime asc"
wtdata = pd.read_sql_query(wtdta, myconn, parse_dates = "Time")
wtdata.rename(columns = {'ftime':'Time', 'weight':'Weight' }, inplace = True)
   
fig4, ax4, = plt.subplots(1, 2)
plt.subplot(1,2,1)
# start setting up figure
mylegend = "7 days stats "
mystats = wtdata.describe(include='float')
plt.ylim(230,260)
ax4[0].set_xlabel('Date')
plt.title('weight plot')
ax4[0].annotate([mystats], xy=(200, 380), xycoords='figure points')
plt.setp(ax4[0].get_xticklabels(), rotation = 90, fontsize=6)
# ax4.set_xticklabels(wtdata.bsdate, rotation=90, fontsize=6)
plt.grid(visible=True, which='both', axis='both', )
fig4.set_figwidth(15)
fig4.set_figheight(10)
# thetable = pd.plotting.table(fig4, wtdata, colLabels= wtdata.columns )
lines = ax4[0].plot(wtdata.Time , wtdata.Weight, marker='o', linestyle='dashed' )
plt.subplot(1,2,2)
plt.table(cellText=wtdata.values,colWidths = [0.25]*len(wtdata.columns),
          rowLabels= None ,
          colLabels=wtdata.columns,
          cellLoc = 'center', rowLoc = 'center',
          loc='center')
mplcursors.cursor(lines) # or just mplcursors.cursor()
plt.show()


# Edit for lineplot only

## Barplots averages for sugar

In [ ]:
sugnext8 = "SELECT bsdate,bsugar FROM qtsugar WHERE bsdate < (SELECT date('now','-8 day')) and bsdate > (SELECT date('now' , '-16 day'))"
sugar8next = pd.read_sql_query(sugnext8, myconn, parse_dates = "bsdate")

In [ ]:
sugeightdays = "SELECT bsdate,bsugar FROM qtsugar WHERE bsdate > (SELECT date('now','-8 day'))"
sugar8days = pd.read_sql_query(sugeightdays, myconn, parse_dates = "bsdate")

In [ ]:
sugtoday = "SELECT bsdate,bsugar FROM qtsugar WHERE bsdate = (SELECT date('now', '-1 day'))"
sugartoday = pd.read_sql_query(sugeightdays, myconn, parse_dates = "bsdate")

In [ ]:



sugonedays = "SELECT bsdate,bsugar FROM qtsugar WHERE bsdate >= (SELECT date('now', '-24 hours'))" 
sugar1days = pd.read_sql_query(sugonedays, myconn, parse_dates = "bsdate")





# Draw a vertical bar chart

plot = sugar1days.plot.bar(x="bsdate", y="bsugar", title="sugar 24 hours")

plt.tight_layout()
plt.show()

In [ ]:
bp8days = "SELECT bpdate ,bpsys AS systolic, bpdia AS diastolic, bphr AS pulse , bpsugar as glucose, ketone, bpcomment as comment from  vsigns_bp where bpdate > (SELECT date('now','-8 day'))"
pd.read_sql_query(bp8days, myconn, parse_dates = "bpdate")


## Make a report

Use DataFrame. columns. values. tolist() 

In [ ]:
def df2table(df):
    return Table(
      [[Paragraph(col) for col in df.columns]] + df.values.tolist(), 
      style=[
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('LINEBELOW',(0,0), (-1,0), 1, colors.black),
        ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
        ('BOX', (0,0), (-1,-1), 1, colors.black),
        ('ROWBACKGROUNDS', (0,0), (-1,-1), [colors.lightgrey, colors.white])],
      hAlign = 'LEFT')

In [ ]:
try:
    with conn.cursor() as cursor:
        # Read data from database
        sql = "SELECT * FROM `users`"
        cursor.execute(sql)

        # Fetch all rows
        rows = cursor.fetchall()

        # Print results
        for row in rows:
            print(row)
finally:
    conn.close()

In [ ]:
from reportlab.pdfgen.canvas import Canvas
from reportlab.platypus import SimpleDocTemplate, Table, Paragraph
from reportlab.platypus import TableStyle
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.styles import getSampleStyleSheet
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
bp8days = "SELECT bpdate ,bpsys AS systolic, bpdia AS diastolic, bphr AS pulse , bpsugar as glucose, ketone, bpcomment as comment from  vsigns_bp where bpdate > (SELECT date('now','-3 day'))"

# bpdf = pd.read_sql_query(bp8days, myconn, parse_dates = "bpdate")
bpdf = pd.read_sql_query(bp8days, myconn, )
bpsession = sessionmaker(bind=eng)
bps1 = bpsession()

#bps = bpsession.execute("SELECT bpdate ,bpsys AS systolic, bpdia AS diastolic, bphr AS pulse , bpsugar as glucose, ketone, bpcomment as comment from  vsigns_bp where bpdate > (SELECT date('now','-3 day'))")
# bpsr = bps1.fetchall()
rows = text(bps1.execute(bp8days).all())

stylesheet=getSampleStyleSheet()
normalStyle = stylesheet['Normal']

bpdata =["DATE" , "SYSTOLIC", "DIASTOLIC", "PULSE", "GLUCOSE", "KETONE", "COMMENT"]

#bpdata.append = (bpdf.values)


#for i in range(1,100):
    #bpdata.append([str(i),str(i),str(i),str(i),str(i),str(i),str(i)])
    #bpdata.append([(np.array(bpdf).tolist())])
bpdata.append((np.array(bpdf)))
#sugar1header = ''.join(str(e) for e in sugar1header)
# s1p = Paragraph(sugar1header,normalStyle)
tstx = "test text"
s1p = Paragraph(tstx,normalStyle,bulletText=None)

c1 = Canvas('table.pdf', pagesize=letter)
GRID_STYLE = TableStyle(
            [('GRID', (0, 0), (-1, -1), 0.25, colors.green),
            ('ALIGN', (1, 0), (-1, -1), 'RIGHT')])
#tbuild = [("DATE" , "SYSTOLIC", "DIASTOLIC", "PULSE", "GLUCOSE", "KETONE", "COMMENT")]

#tbuild.append(bpdf)
t1 = Table(np.array(bpdf).tolist())
#t1 = Table(bpdata)

#t1 = df2table(bpdf)


t1.setStyle(GRID_STYLE) 
# doc = SimpleDocTemplate("table.pdf", pagesize=letter)

# Draw the table on the canvas
t1.wrapOn(c1, 100, 500)
t1.drawOn(c1, 100, 500)
# c1.drawString(100, 700, sugar1header)
#s1p.drawOn(c1,100, 600)
#s1p.wrapOn(c1,100,80)
#sugar1header.drawOn(c1,100,80)
# Save the canvas
c1.save()



In [ ]:
bps1


In [ ]:
# from reportlab.pdfgen.canvas import Canvas
# import the canvas object
from reportlab.pdfgen import canvas

# create a Canvas object with a filename
c = canvas.Canvas("rt.pdf", pagesize=(595.27, 841.89))  # A4 pagesize
# draw a string at x=100, y=800 points
# point ~ standard desktop publishing (72 DPI)
# coordinate system:
#   y
#   |
#   |   page
#   |
#   |
#   0-------x
#c.drawString(50, 780, "Hello Again")
c.drawString(50,780, sugar1days)
# finish page
c.showPage()
# construct and save file to .pdf
c.save()


In [ ]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Table, TableStyle, Paragraph
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.styles import getSampleStyleSheet

# Create a table
data = [
    ['Name', 'Age', 'City'],
    ['Alice', 25, 'London'],
    ['Bob', 30, 'Paris'],
    ['Charlie', 35, 'Rome'],
]
table = Table(data)

# Create a table style
style = TableStyle([
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.black),
    ('BOX', (0, 0), (-1, -1), 0.25, colors.black),
])

# Apply the table style to the table
table.setStyle(style)

# Create a canvas
canvas = Canvas('table.pdf', pagesize=letter)

# Draw the table on the canvas
table.wrapOn(canvas, 100, 100)
table.drawOn(canvas, 100, 100)

# Save the canvas
canvas.save()

In [7]:
import io
import os
# from django.core.files.base import ContentFile
# from jsignature.utils import draw_signature
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_RIGHT, TA_CENTER, TA_LEFT
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table
from PIL import Image


# create pdf with table and paragraphs
def create_pdf(participant):
    # create a file-like buffer to receive PDF data
    buffer = io.BytesIO()
    # define styles
    styles = getSampleStyleSheet()
    style_general = ParagraphStyle(
        name='left',
        parent=styles['Normal'],
        fontSize=12,
        fontName='Helvetica',
        alignment=TA_LEFT)
    style_image = ParagraphStyle(
        name='left',
        fontSize=30,
        parent=styles['Normal'],
        alignment=TA_LEFT)
    style_heading = ParagraphStyle(
        name='center',
        fontSize=18,
        fontName='Helvetica-Bold',
        parent=styles['Heading1'],
        leading=18,
        alignment=TA_CENTER)
    # create a simple document with page size in buffer
    doc = SimpleDocTemplate(buffer, pagesize=letter, author='Me')
    # create a list of paragraphs
    AllParagraphs = []
    # convert png image to jpeg
    jpeg_image = get_jpeg_image(participant)
    # add rows and columns so that the data can align
    table_data = [
        [Paragraph("My Heading - It should span the full page width", style_heading)],
        [
            Paragraph('Name:', style_general),
            Paragraph(
                f'{participant.first_name} {participant.middle_initial} {participant.last_name}',
                style_general)
        ],
        [
            Paragraph(f'Signature:', style_general),
            # image height of 30 to prevent overlapping since fontSize is 30,
            # image width of double to maintain aspect ratio
            Paragraph(
                "<img src='{0}' valign='middle' width=60 height=30 />".format(
                    jpeg_image),
                style_image)
        ]
    ]
    # set rows and columns into Table object
    table_element = Table(table_data)
    # add table to list of paragraphs
    AllParagraphs.append(table_element)
    # build document with list of paragraphs
    doc.build(AllParagraphs)

    # get content of buffer
    buffer.seek(0)
    pdf_data = buffer.getvalue()
    # save buffer content to django File object
    file_data = ContentFile(pdf_data)
    # name pdf file
    file_data.name = f'{participant.last_name}.pdf'
    # delete jpeg file
    os.remove(jpeg_image)
    # save pdf file to parent model
    participant.pdf = file_data
    participant.save()


In [8]:
pdf_data

NameError: name 'pdf_data' is not defined